In [1]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
nltk.download('treebank')
nltk_data = list(nltk.corpus.treebank.tagged_sents())

[nltk_data] Downloading package treebank to /Users/ting/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
# let's check some of the tagged data
print(nltk_data[0])

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]


In [4]:
# split data into training and validation set in the ratio 95:5
train_set,test_set = train_test_split(nltk_data,train_size=0.8,test_size=0.2,random_state = 0)

# create list of train and test tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
print(len(train_tagged_words))
print(len(test_tagged_words))

# check some of the tagged words.
print(train_tagged_words[0:5])

# let's check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

# let's check how many words are present in vocabulary
vocab = {word for word,tag in train_tagged_words}
print(len(vocab))

# compute emission probability for a given word for a given tag
def word_given_tag(word,tag,train_bag= train_tagged_words):
    taglist = [pair for pair in train_bag if pair[1] == tag]
    tag_count = len(taglist)    
    w_in_tag = [pair[0] for pair in taglist if pair[0]==word]    
    word_count_given_tag = len(w_in_tag)    
    
    return (word_count_given_tag,tag_count)

# compute transition probabilities of a previous and next tag
def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    
    t1_tags = [tag for tag in tags if tag==t1]
    
    count_of_t1 = len(t1_tags)
    
    t2_given_t1 = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    
    count_t2_given_t1 = len(t2_given_t1)
    
    return(count_t2_given_t1,count_of_t1)

t2_given_t1('NOUN','DET')

# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
        
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

79930
20746
[('Edward', 'NNP'), ('L.', 'NNP'), ('Kane', 'NNP'), ('succeeded', 'VBD'), ('Mr.', 'NNP')]
46
{'-LRB-', 'PDT', 'FW', 'CC', 'LS', 'JJS', 'VBP', '-RRB-', 'JJ', 'RBS', 'RBR', 'NNPS', 'NNP', 'WRB', 'WP$', '.', 'NNS', '``', 'PRP', 'VB', 'VBD', 'VBZ', "''", 'RP', 'EX', 'MD', 'POS', ':', '#', 'CD', 'NN', 'VBG', 'JJR', 'VBN', 'PRP$', 'RB', 'WP', '$', 'IN', '-NONE-', 'TO', 'UH', 'WDT', ',', 'DT', 'SYM'}
10958


In [5]:
tags_df

,-LRB-,PDT,FW,CC,LS,JJS,VBP,-RRB-,JJ,RBS,...,WP,$,IN,-NONE-,TO,UH,WDT,",",DT,SYM
-LRB-,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,...,0.000000,0.161290,0.064516,0.010753,0.000000,0.000000,0.000000,0.000000,0.096774,0.000000
PDT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.869565,0.000000
FW,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CC,0.000000,0.000000,0.000000,0.000556,0.000556,0.002782,0.011130,0.000556,0.093489,0.000000,...,0.001669,0.020033,0.053422,0.006678,0.005565,0.000000,0.000556,0.008347,0.115192,0.000000
LS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
JJS,0.000000,0.000000,0.000000,0.007042,0.000000,0.000000,0.007042,0.000000,0.147887,0.000000,...,0.000000,0.000000,0.154930,0.014085,0.000000,0.000000,0.000000,0.007042,0.000000,0.000000
VBP,0.000000,0.000948,0.000000,0.003791,0.000000,0.000000,0.000000,0.000000,0.071090,0.000000,...,0.000948,0.002844,0.096682,0.158294,0.009479,0.000000,0.000948,0.007583,0.104265,0.000000
-RRB-,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.010417,0.000000,0.020833,0.000000,...,0.000000,0.000000,0.093750,0.041667,0.010417,0.000000,0.000000,0.208333,0.083333,0.000000
JJ,0.000219,0.000000,0.000000,0.015145,0.000000,0.000219,0.000878,0.000219,0.063433,0.000000,...,0.000219,0.001756,0.055751,0.021291,0.011633,0.000000,0.000000,0.029192,0.003292,0.000000
RBS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.709677,0.000000,...,0.000000,0.000000,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
#Viterbi Algorithm
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [7]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sentences
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sentences
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [8]:
# tagging the test sentences
tagged_seq = Viterbi(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Viterbi Algorithm Accuracy:  86.8421052631579


In [9]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('broke', '-LRB-'), ('broke', 'VBD')),
 (('my', '-LRB-'), ('my', 'PRP$')),
 (('cranked', '-LRB-'), ('cranked', 'VBD')),
 (('up', 'IN'), ('up', 'RP')),
 (('worthy', '-LRB-'), ('worthy', 'JJ')),
 (('murder', '-LRB-'), ('murder', 'NN')),
 (('incest', '-LRB-'), ('incest', 'NN')),
 ((',', '-LRB-'), (',', ',')),
 (("''", '-LRB-'), ("''", "''")),
 (('dynamics', '-LRB-'), ('dynamics', 'NNS')),
 (('transforming', '-LRB-'), ('transforming', 'VBG')),
 (('packaging', 'VBG'), ('packaging', 'NN')),
 (('130.6', '-LRB-'), ('130.6', 'CD')),
 (('Hiroshi', '-LRB-'), ('Hiroshi', 'NNP')),
 (('Asada', '-LRB-'), ('Asada', 'NNP'))]